# Autonomous Trading Agents: MCP Resource Templates

**Project:** Autonomous equity trading simulation

This notebook demonstrates an advanced multi-agent system powered by MCP resources and prompt templates. The architecture includes:
- **4 Autonomous Traders**: Each with distinct investment strategies
- **1 Research Agent**: Web research and financial data analysis
- **5 MCP Servers**: Account management, market data, memory, notifications, and web search

**Key Pattern:** MCP Resources enable agent personalization through context injection (account state, investment strategies).

In [ ]:
# Import dependencies
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

## Phase 1: Configure MCP Servers

In [ ]:
# Determine Market Data Source
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(f"Paid Plan: {is_paid_polygon}")
print(f"Realtime Plan: {is_realtime_polygon}")

In [ ]:
# Configure Trader MCP Servers
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {
        "command": "uvx",
        "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"],
        "env": {"POLYGON_API_KEY": polygon_api_key}
    }
else:
    market_mcp = {"command": "uv", "args": ["run", "market_server.py"]}

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

In [ ]:
# Configure Researcher MCP Servers
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

In [ ]:
# Initialize MCP Servers
researcher_mcp_servers = [
    MCPServerStdio(params, client_session_timeout_seconds=30) 
    for params in researcher_mcp_server_params
]
trader_mcp_servers = [
    MCPServerStdio(params, client_session_timeout_seconds=30) 
    for params in trader_mcp_server_params
]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

## Phase 2: Build Research Agent Tool

In [ ]:
# Research Agent Factory
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. Search the web for news and trading opportunities.
Based on requests, conduct thorough research and summarize findings.
If no specific request, search for investment opportunities in latest news.
Current datetime: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    return Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )

async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
        tool_name="Researcher",
        tool_description="""Research financial news and opportunities online. 
Specify what you want researched, or request general market opportunities."""
    )

In [ ]:
# Test Researcher
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()

researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))

## Phase 3: Build Trader Agent with Resources

In [ ]:
# Initialize Trader Account
agent_name = "TestTrader"
initial_strategy = "Day trader: aggressively buy/sell based on news and market conditions."
Account.get(agent_name).reset(initial_strategy)

# Display Account State
display(Markdown(await read_accounts_resource(agent_name)))
display(Markdown(await read_strategy_resource(agent_name)))

In [ ]:
# Build Trader Instructions (MCP Resource Injection)
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""You are a trader managing a portfolio. Your name is {agent_name}, account under {agent_name}.
You have tools to:
- Search internet for company news
- Check stock prices
- Buy/sell shares
- Save memory of companies and research

**Investment Strategy:**
{strategy}

**Current Holdings:**
{account_details}

Use your tools to manage your portfolio autonomously. Execute trades without asking for confirmation.
"""

prompt = """Investigate news and market conditions, make trading decisions, execute trades, 
and summarize your actions."""

In [ ]:
# Execute Trader
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)

with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

In [ ]:
# View Trading Results
await read_accounts_resource(agent_name)

## Phase 4: Production Implementation

**Architecture Files:**
- `mcp_params.py`: MCP server configurations
- `templates.py`: Instruction and prompt templates
- `traders.py`: Trader agent implementation
- `reset.py`: Initialize trader accounts with strategies

**Code Pattern:** Using `AsyncExitStack` for clean resource management:
```python
async with AsyncExitStack() as stack:
    mcp_servers = [
        await stack.enter_async_context(MCPServerStdio(params)) 
        for params in mcp_server_params
    ]
```

In [ ]:
# Use Production Trader Module
from traders import Trader

trader = Trader("TestTrader")
await trader.run()

In [ ]:
# Check Results
await read_accounts_resource("TestTrader")

## Tool Count Analysis

In [ ]:
# Calculate Total Tools
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("test")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)

print(f"We have {len(all_params)} MCP servers and {count} total tools")